In [1]:
import requests, pandas as pd
from bs4 import BeautifulSoup

def fetch_wikipedia_reactors():
    url = "https://en.wikipedia.org/wiki/List_of_nuclear_reactors"
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    # find the right table, parse into df...
    tables = pd.read_html(r.text)
    reactors = tables[0]  # adjust index as needed
    reactors.to_csv("data/reactors.csv", index=False)
    return reactors


In [2]:
fetch_wikipedia_reactors()

C:\Users\Armstrong\AppData\Local\Temp\ipykernel_22380\3949624661.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(r.text)


,vteTypes of nuclear fission reactor,vteTypes of nuclear fission reactor.1
0,Moderator,Moderator
1,Light water,Aqueous homogeneous Boiling BWR ABWR ESBWR Ker...
2,Heavy water by coolant,D2O Pressurized CANDU CANDU 6 CANDU 9 EC6 AFCR...
3,D2O,Pressurized CANDU CANDU 6 CANDU 9 EC6 AFCR ACR...
4,H2O,HWLWR ATR HW BLWR 250 Steam-generating (SGHWR)...
5,Organic,WR-1
6,CO2,HWGCR EL-4 KKN KS 150 Lucens
7,Graphite by coolant,WaterH2O AM-1 AMB-X EGP-6 RBMK MKER GasCO2 Ura...
8,Water,H2O AM-1 AMB-X EGP-6 RBMK MKER
9,H2O,AM-1 AMB-X EGP-6 RBMK MKER


In [7]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import os
import time

def get_country_slugs():
    """
    Fetches the country slugs from the World Nuclear Association Reactor Database summary page.
    """
    url = 'https://world-nuclear.org/nuclear-reactor-database/summary'
    headers = {'User-Agent': 'Mozilla/5.0'}
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')
    
    # Locate the country dropdown/select element
    select = soup.find('select')
    slugs = []
    for option in select.find_all('option'):
        slug = option.get('value', '').strip()
        if slug:
            slugs.append(slug)
    return slugs

def fetch_country_summary(slug, headers):
    """
    Fetches and concatenates all tables from a specific country's summary page.
    """
    url = f'https://world-nuclear.org/nuclear-reactor-database/summary/{slug}'
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    tables = pd.read_html(resp.text)
    df = pd.concat(tables, ignore_index=True)
    df['Country'] = slug.replace('%20', ' ')
    return df

def scrape_all_countries(output_csv='data/wn_all_countries_reactors.csv'):
    """
    Scrapes reactor data for all countries and saves combined CSV.
    """
    # Ensure data directory exists
    os.makedirs(os.path.dirname(output_csv), exist_ok=True)
    
    headers = {'User-Agent': 'Mozilla/5.0'}
    slugs = get_country_slugs()
    all_dfs = []
    
    for slug in slugs:
        try:
            df = fetch_country_summary(slug, headers)
            all_dfs.append(df)
            print(f"Fetched data for {slug}")
        except Exception as e:
            print(f"Error fetching {slug}: {e}")
        time.sleep(1)  # Be polite to the server
    
    combined = pd.concat(all_dfs, ignore_index=True)
    combined.to_csv(output_csv, index=False)
    return combined

if __name__ == "__main__":
    df_all = scrape_all_countries()
    print("Combined dataset:")
    print(df_all.head())



C:\Users\Armstrong\AppData\Local\Temp\ipykernel_22380\4262000673.py:33: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(resp.text)


Fetched data for Argentina


C:\Users\Armstrong\AppData\Local\Temp\ipykernel_22380\4262000673.py:33: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(resp.text)


Fetched data for Armenia


C:\Users\Armstrong\AppData\Local\Temp\ipykernel_22380\4262000673.py:33: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(resp.text)


...

Fetched data for United Arab Emirates


C:\Users\Armstrong\AppData\Local\Temp\ipykernel_22380\4262000673.py:33: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(resp.text)


Fetched data for United Kingdom


C:\Users\Armstrong\AppData\Local\Temp\ipykernel_22380\4262000673.py:33: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(resp.text)


Fetched data for United States Of America
Combined dataset:
  Reactor Name                             Model Process  Capacity (MWe)  \
0     Atucha 1                          PHWR KWU    PHWR           340.0   
1     Atucha 2                          PHWR KWU    PHWR           693.0   
2      Embalse                           CANDU 6    PHWR           608.0   
3      Carem25  CAREM Prototype (Integrated-PWR)     PWR            25.0   
4   Armenian 2                        VVER V-270     PWR           416.0   

  Grid Connection Load Factor (2023) (%) Electricity Generated (2023) (GWh)  \
0      19/03/1974                   73.8                            2197.12   
1      27/06/2014                   25.4                            1541.93   
2      25/04/1983                   98.1                            5224.06   
3             NaN                    NaN                                NaN   
4      05/01/1980                   68.9                            2512.03   

  Constr

In [10]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

def fetch_uranium_production_cleaned(output_csv: str) -> pd.DataFrame:
    """
    Scrapes and cleans the 'Uranium production figures' table from WNA.
    Handles repeated headers and non-numeric rows gracefully.
    """
    url = "https://world-nuclear.org/Information-Library/Facts-and-Figures/Uranium-production-figures"
    headers = {"User-Agent": "Mozilla/5.0"}
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()

    # Parse all tables; pick the one with 'Country' in the first column
    tables = pd.read_html(resp.text)
    df = None
    for tbl in tables:
        cols = [str(c).strip() for c in tbl.columns]
        if 'Country' in cols:
            df = tbl.copy()
            break
    if df is None:
        raise ValueError("Could not locate the uranium production table.")

    # Drop any rows where Country = 'Country' (repeated header rows)
    df = df[df['Country'] != 'Country'].reset_index(drop=True)

    # Clean numeric columns: all except 'Country'
    for col in df.columns:
        if col == 'Country':
            continue
        # Remove commas, strip, coerce to numeric, fill NaN with 0
        df[col] = (
            df[col]
            .astype(str)
            .str.replace(",", "", regex=False)
            .str.strip()
            .replace("", "0")
        )
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

    # Save cleaned data
    df.to_csv(output_csv, index=False)
    return df

# Example usage:
if __name__ == "__main__":
    df_uranium = fetch_uranium_production_cleaned("data/uranium_production_cleaned.csv")
    print("Cleaned Uranium production data:")
    print(df_uranium.head())



Cleaned Uranium production data:
             Country   2013   2014   2015   2016   2017   2018   2019   2020  \
0         Kazakhstan  22451  23127  23607  24689  23321  21705  22808  19477   
1             Canada   9331   9124  13325  14039  13116   7001   6938   3885   
2            Namibia   4323   3255   2993   3654   4224   5525   5476   5413   
3          Australia   6350   5001   5654   6315   5882   6517   6613   6203   
4  Uzbekistan (est.)   2400   2400   2385   3325   3400   3450   3500   3500   

    2021   2022  
0  21819  21227  
1   4693   7351  
2   5753   5613  
3   4192   4553  
4   3520   3300  


C:\Users\Armstrong\AppData\Local\Temp\ipykernel_22380\818903822.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(resp.text)


In [11]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

def fetch_nuclear_generation_by_country(output_csv: str) -> pd.DataFrame:
    """
    Scrapes the 'Nuclear generation by country' table from WNA and saves it to CSV.
    """
    url = "https://world-nuclear.org/Information-Library/Facts-and-Figures/nuclear-generation-by-country"
    headers = {"User-Agent": "Mozilla/5.0"}
    
    # Attempt to fetch page
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()

    # Parse all tables with pandas
    tables = pd.read_html(resp.text)
    
    # Identify the correct table by looking for 'Country or area' in the columns
    for tbl in tables:
        cols = [str(c).strip() for c in tbl.columns.get_level_values(0)]
        if "Country or area" in cols:
            df = tbl.copy()
            break
    else:
        raise ValueError("Could not find the nuclear generation table.")

    # Flatten multi-level header
    # Combine level0 and level1 names, ignoring NaNs and 'Unnamed'
    new_cols = []
    for level0, level1 in df.columns:
        header = str(level0).strip()
        if pd.notna(level1) and not str(level1).startswith("Unnamed"):
            header += f" {level1}".strip()
        new_cols.append(header)
    df.columns = new_cols

    # Rename the first column for simplicity
    df = df.rename(columns={"Country or area": "Country"})

    # Clean numeric columns: remove commas, convert to numeric
    for col in df.columns:
        if col == "Country":
            continue
        df[col] = (
            df[col]
            .astype(str)
            .str.replace(",", "", regex=False)
            .str.strip()
            .replace("", "0")
        )
        df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0)

    # Save to CSV
    df.to_csv(output_csv, index=False)
    return df

# Example usage
if __name__ == "__main__":
    df_gen = fetch_nuclear_generation_by_country("data/nuclear_generation_by_country.csv")
    print("Scraped Nuclear Generation by Country:")
    print(df_gen.head(10))


Scraped Nuclear Generation by Country:
      Country  Nuclear share of electricity (%)2013  \
0   Argentina                                   4.4   
1     Armenia                                  29.2   
2     Belarus                                   0.0   
3     Belgium                                  52.1   
4      Brazil                                   2.8   
5    Bulgaria                                  30.7   
6      Canada                                  16.0   
7      China:                                   0.0   
8  - Mainland                                   2.1   
9    - Taiwan                                  19.1   

   Nuclear share of electricity (%)2014  Nuclear share of electricity (%)2015  \
0                                   4.0                                   4.8   
1                                  30.7                                  34.5   
2                                   0.0                                   0.0   
3                              

C:\Users\Armstrong\AppData\Local\Temp\ipykernel_22380\3082047454.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(resp.text)


In [19]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

def fetch_uranium_enrichment_capacity(output_csv: str, html_path: str = None) -> pd.DataFrame:
    """
    Scrapes the uranium enrichment capacity table from WNA, handling multi-index headers.
    If html_path is provided, reads from local file instead of HTTP.
    """
    if html_path:
        with open(html_path, "r", encoding="utf-8") as f:
            html = f.read()
    else:
        url = "https://world-nuclear.org/information-library/nuclear-fuel-cycle/conversion-enrichment-and-fabrication/uranium-enrichment"
        headers = {"User-Agent": "Mozilla/5.0"}
        resp = requests.get(url, headers=headers, timeout=15)
        resp.raise_for_status()
        html = resp.text

    # Parse all tables
    tables = pd.read_html(html)
    selected_df = None

    for tbl in tables:
        # Flatten columns
        cols = tbl.columns
        flat_cols = []
        if isinstance(cols, pd.MultiIndex):
            for top, sub in cols:
                top = str(top).strip()
                sub = str(sub).strip()
                if sub and not sub.startswith("Unnamed"):
                    header = f"{top} {sub}"
                else:
                    header = top
                flat_cols.append(header)
        else:
            flat_cols = [str(c).strip() for c in cols]

        # Check if this is the correct table
        if "Operator" in flat_cols and "2022" in flat_cols:
            df = tbl.copy()
            df.columns = flat_cols
            selected_df = df
            break

    if selected_df is None:
        # For debugging: list column sets
        for i, tbl in enumerate(tables):
            print(f"Table {i} cols:", tbl.columns)
        raise ValueError("Could not find the uranium enrichment capacity table.")

    # Clean numeric columns
    for col in selected_df.columns:
        if col == "Operator":
            continue
        selected_df[col] = (
            selected_df[col]
            .astype(str).str.replace(",", "", regex=False).str.strip()
            .replace("", "0")
        )
        selected_df[col] = pd.to_numeric(selected_df[col], errors="coerce").fillna(0).astype(int)

    selected_df.to_csv(output_csv, index=False)
    return selected_df

# Example usage:
if __name__ == "__main__":
    # Option 1: Direct HTTP
    # df_enrich = fetch_uranium_enrichment_capacity("data/uranium_enrichment_capacity.csv")
    # Option 2: Local HTML fallback
    df_enrich = fetch_uranium_enrichment_capacity(
        "data/uranium_enrichment_capacity.csv",
        html_path=r"C:\Users\Armstrong\nuclear\notebooks\uranium_enrichment.html"
    )
    print(df_enrich)


Table 0 cols: MultiIndex([(                  'Operator', 'Operator'),
            ('Capacity (thousand SWU/yr)',     '2022'),
            ('Capacity (thousand SWU/yr)',     '2025'),
            ('Capacity (thousand SWU/yr)',     '2030')],
           )


C:\Users\Armstrong\AppData\Local\Temp\ipykernel_22380\1363972361.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)


ValueError: Could not find the uranium enrichment capacity table.

In [20]:
import io
import pandas as pd
from bs4 import BeautifulSoup

# Path to the locally saved HTML file (adjust if needed)
html_path = r"C:\Users\Armstrong\nuclear\notebooks\uranium_enrichment.html"

# 1) Load the HTML content
with open(html_path, "r", encoding="utf-8") as f:
    html = f.read()

# 2) Use BeautifulSoup to locate the section for enrichment capacity
soup = BeautifulSoup(html, "html.parser")

# Try finding a header that mentions "Enrichment"
header = soup.find(lambda tag: tag.name in ["h2", "h3"] and "Enrichment" in tag.get_text())
if header:
    # Get the first table following that header
    table_tag = header.find_next("table")
    if table_tag:
        # Parse that table
        df_enrich = pd.read_html(io.StringIO(str(table_tag)))[0]
        print("Enrichment table located via header search:")
        print(df_enrich.head())
    else:
        print("Header found but no subsequent <table> tag present.")
else:
    print("No header containing 'Enrichment' found.")

# 3) As a fallback, list all tables with their index, shape, and first rows
print("\nFallback: Listing all parsed tables:")
tables = pd.read_html(io.StringIO(html))
for idx, tbl in enumerate(tables):
    print(f"\nTable {idx} (shape {tbl.shape}):")
    print(tbl.head())


Enrichment table located via header search:
            Operator Capacity (thousand SWU/yr)              
            Operator                       2022   2025   2030
0               CNNC                       8900  10000  17000
1              Orano                       7500   7500   7500
2            Rosatom                      27100  27100  27100
3             Urenco                      17900  17900  17900
4  Other (INB, JNFL)                        100    400    800

Fallback: Listing all parsed tables:

Table 0 (shape (6, 4)):
            Operator Capacity (thousand SWU/yr)              
            Operator                       2022   2025   2030
0               CNNC                       8900  10000  17000
1              Orano                       7500   7500   7500
2            Rosatom                      27100  27100  27100
3             Urenco                      17900  17900  17900
4  Other (INB, JNFL)                        100    400    800


In [27]:
import io
import pandas as pd
from bs4 import BeautifulSoup

def fetch_uranium_enrichment_capacity_clean(output_csv: str, html_path: str):
    """
    Scrapes and cleans the uranium enrichment capacity table, dropping repeated headers.
    Reads from local HTML file at html_path.
    """
    # 1) Read HTML
    with open(html_path, "r", encoding="utf-8") as f:
        html = f.read()
    
    # 2) Parse table following "Enrichment" header
    soup = BeautifulSoup(html, "html.parser")
    header = soup.find(lambda tag: tag.name in ["h2", "h3"] and "Enrichment" in tag.get_text())
    if not header:
        raise ValueError("Enrichment header not found")
    table_tag = header.find_next("table")
    if table_tag is None:
        raise ValueError("No table found after Enrichment header")

    # 3) Read into DataFrame
    df = pd.read_html(io.StringIO(str(table_tag)))[0]

    # 4) Drop any repeated header rows
    df = df[df.iloc[:, 0] != "Operator"].reset_index(drop=True)

    # 5) Flatten any MultiIndex columns
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [
            f"{str(a).strip()} {str(b).strip()}".strip()
            for a, b in df.columns
        ]

    # 6) Standardize columns
    df.columns = [str(c).strip().replace("\n", " ") for c in df.columns]

    # 7) Convert numeric columns, skipping the first column
    for col in df.columns[1:]:
        df[col] = (
            df[col].astype(str)
            .str.replace(",", "", regex=False)
            .str.strip()
            .replace("", "0")
        )
        df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0).astype(int)

    # 8) Save cleaned CSV
    df.to_csv(output_csv, index=False)
    return df

# Example usage:
if __name__ == "__main__":
    df_clean = fetch_uranium_enrichment_capacity_clean(
        output_csv="data/uranium_enrichment_capacity_clean.csv",
        html_path="data/uranium_enrichment.html"
    )
    print(df_clean)


FileNotFoundError: [Errno 2] No such file or directory: 'data/uranium_enrichment.html'

In [25]:
DATASETS = {
    "OperationalReactorsByCountry":        "Operational Reactors by Country",
    "OperationalReactorsByType":           "Operational Reactors by Type",
    "OperationalReactorsByRegion":         "Operational Reactors by Region",
    "UnderConstructionReactorsByCountry":  "Under Construction by Country",
    "PermanentShutdownReactorsByCountry":  "Permanent Shutdown by Country",
    "Last3YearEnergyAvailability":         "Last 3-Year Energy Availability",
    "Last3YearUnitCapability":             "Last 3-Year Unit Capability",
    "Last3YearUnplannedCapacityLoss":      "Last 3-Year Unplanned Capacity Loss",
    "LifeTimeEnergyAvailability":          "Lifetime Energy Availability",
    "LifeTimeUnitCapability":              "Lifetime Unit Capability",
    "LifeTimeUnplannedCapacityLoss":       "Lifetime Unplanned Capacity Loss",
    "TrendNuclearPowerCapacity":           "Trend Nuclear Power Capacity",
    "TrendEnergyAvailability":             "Trend Energy Availability",
    "TrendUnitCapability":                 "Trend Unit Capability",
    "TrendUnplannedCapacityLoss":          "Trend Unplanned Capacity Loss",
    "TrendLoadFactor":                     "Trend Load Factor",
    "TrendElectricitySupplied":            "Trend Electricity Supplied",
    "TrendNuclearShare":                   "Trend Nuclear Share",
    "TrendAgeDistribution":                "Trend Age Distribution"
}

In [ ]:
https://pris.iaea.org/rdl/odata/PRIS/OperationalReactorsByRegion?$format=json&$top=5000
